In [192]:
import pandas as pd 
import numpy as np 
import sklearn 

In [171]:
J1939 = pd.read_csv('data/J1939Faults.csv')

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [32]:
vehicle = pd.read_csv('data/VehicleDiagnosticOnboardData.csv')

In [38]:
codes = pd.read_excel('data/Service Fault Codes_1_0_0_167.xlsx')

In [33]:
J1939.shape

(1187335, 20)

In [34]:
vehicle.shape

(12821626, 4)

In [36]:
J1939 = J1939.drop(columns = ['actionDescription', 'ecuSoftwareVersion', 'ecuSerialNumber', 'faultValue', 'ecuModel', 'ecuMake'])

In [ ]:
codes.drop(columns = ['Published in CES 14602'])

In [40]:

# merge on RecordID and FaultId


In [41]:
# SPN 5246 is bad 

In [42]:
faults_onboard = pd.merge(left = J1939, right = vehicle, left_on = 'RecordID', right_on = 'FaultId', how = 'inner')

In [44]:
faults_onboard.shape

(12821626, 18)

In [45]:
# filter down to failure rates and get truck ids and then we can try to make a function 
# to figure out what the codes were right before the failure 
# filter the SPNs and get the equipment ID 

In [76]:
df1569 = faults_onboard[faults_onboard['spn'] == 1569]
df1569.EquipmentID.unique()

array(['1721', '1515', '1630', '1395', '1487', '1499', '1329', '1365',
       '1419', '1486', '1509', '1492', '1340', '1328', '1430', '1370',
       '1497', '1350', '1608', '1606', '1612', '1354', '1493', '1583',
       '1465', '1335', '1789', 1340, 1383, 1389, 1501, 1335, 1438, 1463,
       1606, 1514, 1666, 1689, 1692, 1357, 1396, 310, 1602, 1660, 1445,
       1431, 105393153, 1327, 1795, 1399, 1494, 1367, 1364, 1452, 1595,
       1740, 1366, 1339, 1425, 1358, 1646, 1391, 1428, 1338, 1637, 1346,
       1490, 1562, 1433, 1663, 1417, 1424, 1771, 1790, 1591, 1797, 1596,
       1421, 1444, 1480, 1493, 1466, 1457, 105360462, 1794, 1492, 1696,
       1703, 1626, 1715, 1437, 1453, 1584, 1597, 1610, 1377, 1652,
       105411041, 1517, 1784, 1751, 1786, 1590, 1341, 1382, 1714, 1430,
       1711, 1369, 1398, 1791, 1772, 1472, 1473, 1418, 1645, 1475, 1815,
       1806, 1607, 1560, 1450, 1401, 1572, 1630, 1375, 1368, 1650, 1443,
       1495, 1670, 1587, 1778, 1566, 1789, 1617, 1691, 1573, 1477, 

In [53]:
df1569.shape

(114321, 18)

In [54]:
df1569[df1569['fmi'] == 31].shape

(114321, 18)

In [50]:
faults_onboard.EquipmentID.nunique()

1927

In [60]:
df1569.EventTimeStamp.min()

'2000-03-19 14:02:34.000'

In [61]:
df1569.EventTimeStamp.max()

'2020-03-06 14:04:23.000'

In [62]:
faults_onboard.EventTimeStamp.min()

'2000-03-18 19:14:10.000'

In [ ]:
#cdf1569.groupby('EquipmentID')


# if spn ==. , look at EqId, get it, bring in the other rows 

In [82]:
faults_onboard.EquipmentID.dtype

dtype('O')

In [93]:
faults_onboard.EventTimeStamp.iloc[1]

'2015-02-21 10:47:13.000'

In [107]:
faults_onboard.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12821626 entries, 0 to 12821625
Data columns (total 18 columns):
RecordID                 int64
ESS_Id                   int64
EventTimeStamp           object
eventDescription         object
ecuSource                int64
spn                      int64
fmi                      int64
active                   bool
activeTransitionCount    int64
EquipmentID              object
MCTNumber                int64
Latitude                 float64
Longitude                float64
LocationTimeStamp        object
Id                       int64
Name                     object
Value                    object
FaultId                  int64
dtypes: bool(1), float64(2), int64(9), object(6)
memory usage: 2.0+ GB


In [125]:
# Dalila's code: 

failure_error_2 = faults_onboard.spn.isin([5246])

failure_error = faults_onboard.spn.isin([1569]) & faults_onboard.fmi.isin([31])

all_failures = failure_error_2 | failure_error

failed_trucks = faults_onboard[all_failures].EquipmentID.astype(str).unique()

failed_truck_df = faults_onboard[faults_onboard.EquipmentID.isin(failed_trucks)]

In [126]:
failed_truck_df = failed_truck_df.sort_values(["EquipmentID", "EventTimeStamp"], ascending = (False, True))
failed_truck_df.head()

,RecordID,ESS_Id,EventTimeStamp,eventDescription,ecuSource,spn,fmi,active,activeTransitionCount,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp,Id,Name,Value,FaultId
140,20,990544,2015-02-21 11:18:20.000,High Voltage (Fuel Level),49,96,3,True,126,310,105442984,35.485787,-86.466712,2015-02-21 11:47:32.000,141,Throttle,0,20
141,20,990544,2015-02-21 11:18:20.000,High Voltage (Fuel Level),49,96,3,True,126,310,105442984,35.485787,-86.466712,2015-02-21 11:47:32.000,142,IgnStatus,True,20
142,20,990544,2015-02-21 11:18:20.000,High Voltage (Fuel Level),49,96,3,True,126,310,105442984,35.485787,-86.466712,2015-02-21 11:47:32.000,143,EngineOilPressure,28.42,20
143,20,990544,2015-02-21 11:18:20.000,High Voltage (Fuel Level),49,96,3,True,126,310,105442984,35.485787,-86.466712,2015-02-21 11:47:32.000,144,EngineOilTemperature,192.1438,20
144,20,990544,2015-02-21 11:18:20.000,High Voltage (Fuel Level),49,96,3,True,126,310,105442984,35.485787,-86.466712,2015-02-21 11:47:32.000,145,FuelTemperature,32,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10898196,1053024,60660684,2018-09-05 09:01:08.000,Incorrect Data J1939 Network #1 Primary Vehicl...,11,639,2,True,127,105344451,105344451,39.899120,-82.020046,2018-09-05 09:01:44.000,10923927,CruiseControlActive,False,1053024
10898197,1053024,60660684,2018-09-05 09:01:08.000,Incorrect Data J1939 Network #1 Primary Vehicl...,11,639,2,True,127,105344451,105344451,39.899120,-82.020046,2018-09-05 09:01:44.000,10923928,EngineCoolantTemperature,185,1053024
10898198,1053024,60660684,2018-09-05 09:01:08.000,Incorrect Data J1939 Network #1 Primary Vehicl...,11,639,2,True,127,105344451,105344451,39.899120,-82.020046,2018-09-05 09:01:44.000,10923929,ParkingBrake,False,1053024
10898199,1053024,60660684,2018-09-05 09:01:08.000,Incorrect Data J1939 Network #1 Primary Vehicl...,11,639,2,True,127,105344451,105344451,39.899120,-82.020046,2018-09-05 09:01:44.000,10923930,DistanceLtd,482313,1053024


In [127]:
failed_truck_df['EventTimeStamp'] = pd.to_datetime(failed_truck_df['EventTimeStamp'])
failed_truck_df['Year'] = pd.DatetimeIndex(failed_truck_df['EventTimeStamp']).year
failed_truck_df['Month'] = pd.DatetimeIndex(failed_truck_df['EventTimeStamp']).month

1) Delete rows with shop address

2)  Deal with the duplicates -- maybe use J1939

3)  Find the codes just before the derate codes 

4)  k nearest neighbors

Here are the locations of the shop facilities:

193 Aldi Blvd, Mt Juliet, TN: 36.0666667, -86.4347222

137 Eady Road – Bell Buckle, TN: 35.5883333, -86.4438888

5754 Fish Hatchery Road – Russellville, TN: 36.1950, -83.174722

In [132]:
# round both columns 

failed_truck_df['Lat_Round'] = failed_truck_df.Latitude.round(3)
failed_truck_df['Long_Round'] = failed_truck_df.Longitude.round(3)

In [135]:
failed_truck_df = failed_truck_df[~failed_truck_df.Lat_Round.isin([36.066, 36.067, 35.588, 35.589, 36.195, 36.196])]

failed_truck_df = failed_truck_df[~failed_truck_df.Long_Round.isin([-86.434, -86.435, -86.443, -86.444, -83.174, -83.175])]

In [140]:


# Hari said we should go for rolling windows 



diabetes_rolling=diabetes.groupby('patient_nbr').rolling(window='365D', on='admit_date').encounter_id.count()\
                .reset_index().rename(columns = {'encounter_id':'number_inpatient_visit'})

In [141]:
# faults_CMMNS_onboard_drop.loc[faults_CMMNS_onboard_drop['active'] == True, 'active'] = 1

# faults_CMMNS_onboard_drop.loc[(faults_CMMNS_onboard_drop['SPN'] == 5246) & (faults_CMMNS_onboard_drop['J1939 FMI'] == 0), 'derate'] = 1

In [166]:
# target_var = failed_truck_df.loc[(failed_truck_df['spn'] == 5246) & (failed_truck_df['fmi'] == 0), 'Derate'] = 1

In [179]:
J1939.loc[J1939['spn'] == 5246, 'target'] = 1
J1939.loc[J1939['spn'] != 5246, 'target'] = 0
J1939.loc[J1939['spn'] == 1569, 'target'] = 1
J1939.loc[J1939['spn'] != 1569, 'target'] = 0

In [184]:
pd.set_option('display.max_columns', None)
J1939[J1939.target == 1].head()

,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,faultValue,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp,target
40,41,990856,2015-02-21 12:06:22.000,Condition Exists Engine Protection Torque Derate,NaN,05317106*04047493*092613211021*09300006*G1*BDR*,79723635,6X1u13D1500000000,CMMNS,0,1569,31,True,5,NaN,1721,105439606,39.051805,-84.560509,2015-02-21 12:06:57.000,1.0
249,250,993542,2015-02-21 15:01:49.000,Condition Exists Engine Protection Torque Derate,NaN,05317106*04047493*092613211021*09300006*G1*BDR*,79723635,6X1u13D1500000000,CMMNS,0,1569,31,False,5,NaN,1721,105439606,38.113240,-85.667361,2015-02-21 15:01:45.000,1.0
290,291,994045,2015-02-21 15:35:45.000,Condition Exists Engine Protection Torque Derate,NaN,05317106*04047493*092613211021*09300006*G1*BDR*,79723635,6X1u13D1500000000,CMMNS,0,1569,31,True,6,NaN,1721,105439606,37.735185,-85.808101,2015-02-21 15:36:21.000,1.0
306,307,994237,2015-02-21 15:51:58.000,Condition Exists Engine Protection Torque Derate,NaN,05317106*04047493*092613211021*09300006*G1*BDR*,79723635,6X1u13D1500000000,CMMNS,0,1569,31,False,6,NaN,1721,105439606,37.540370,-85.880972,2015-02-21 15:51:54.000,1.0
340,341,994680,2015-02-21 16:22:24.000,Condition Exists Engine Protection Torque Derate,NaN,05317106*04047493*092613211021*09300006*G1*BDR*,79723635,6X1u13D1500000000,CMMNS,0,1569,31,True,7,NaN,1721,105439606,37.166666,-85.964027,2015-02-21 16:23:00.000,1.0


In [185]:
failed_truck_df.loc[failed_truck_df['spn'] == 5246, 'target'] = 1
failed_truck_df.loc[failed_truck_df['spn'] != 5246, 'target'] = 0
failed_truck_df.loc[failed_truck_df['spn'] == 1569, 'target'] = 1
failed_truck_df.loc[failed_truck_df['spn'] != 1569, 'target'] = 0

In [217]:
truck_310 = faults_onboard[faults_onboard.EquipmentID.isin([310])]
truck_310['EventTimeStamp'] = pd.to_datetime(truck_310['EventTimeStamp'])
#truck_310['Year'] = pd.DatetimeIndex(truck_310['EventTimeStamp']).year
#truck_310['Month'] = pd.DatetimeIndex(truck_310['EventTimeStamp']).month

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [189]:
# First find a way to apply the search to one truck, see if it works and then do a for loop
# over J1939. 

In [215]:
truck_310.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12548 entries, 487727 to 12684110
Data columns (total 20 columns):
RecordID                 12548 non-null int64
ESS_Id                   12548 non-null int64
EventTimeStamp           12548 non-null datetime64[ns]
eventDescription         12396 non-null object
ecuSource                12548 non-null int64
spn                      12548 non-null int64
fmi                      12548 non-null int64
active                   12548 non-null bool
activeTransitionCount    12548 non-null int64
EquipmentID              12548 non-null object
MCTNumber                12548 non-null int64
Latitude                 12548 non-null float64
Longitude                12548 non-null float64
LocationTimeStamp        12548 non-null object
Id                       12548 non-null int64
Name                     12548 non-null object
Value                    12548 non-null object
FaultId                  12548 non-null int64
Year                     12548 non-nul

In [216]:
truck_310['hour'] = truck_310['EventTimeStamp'].dt.hour
truck_310['day'] = truck_310['EventTimeStamp'].dt.day
truck_310['month'] = truck_310['EventTimeStamp'].dt.month
truck_310['Year'] = truck_310['EventTimeStamp'].dt.year

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

In [218]:
truck_310.head()

,RecordID,ESS_Id,EventTimeStamp,eventDescription,ecuSource,spn,fmi,active,activeTransitionCount,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp,Id,Name,Value,FaultId
487727,49403,2362799,2015-05-11 12:52:51,Condition Exists Engine Protection Torque Derate,0,1569,31,True,126,310,105442984,35.264259,-86.398425,2015-05-11 12:55:27.000,488970,IgnStatus,False,49403
487728,49403,2362799,2015-05-11 12:52:51,Condition Exists Engine Protection Torque Derate,0,1569,31,True,126,310,105442984,35.264259,-86.398425,2015-05-11 12:55:27.000,488971,LampStatus,2047,49403
487729,49404,2362808,2015-05-11 12:56:19,Condition Exists Engine Protection Torque Derate,0,1569,31,False,126,310,105442984,35.264259,-86.398425,2015-05-11 12:56:14.000,488972,LampStatus,1023,49404
496759,50415,2382785,2015-05-12 11:34:48,High (Severity Medium) J1939 Network #2,49,1231,16,True,4,310,105442984,35.262916,-86.396250,2015-05-12 11:35:24.000,498002,Throttle,0,50415
496760,50415,2382785,2015-05-12 11:34:48,High (Severity Medium) J1939 Network #2,49,1231,16,True,4,310,105442984,35.262916,-86.396250,2015-05-12 11:35:24.000,498003,IgnStatus,True,50415


In [213]:
pd.Period(value = '12:52:51.000', hour = 6)

Period('0001-01-01 12:52:51', 'S')

# Messing Around

In [78]:
for x in range: 
    group = faults_onboard[faults_onboard.EquipmentID == x]
    
    def prev_code(): 
        if faults_onboard.spn.isin([1569|5246])
        
        return group.spn.index -1

SyntaxError: invalid syntax (<ipython-input-78-428cbcdb837b>, line 5)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(data.iloc[:,0:4], data['Name'])

# Predicted class
print(neigh.predict(test))

-> ['Iris-virginica']

# 3 nearest neighbors
print(neigh.kneighbors(test)[1])
-> [[141 139 120]]



In [186]:
%paste
def is_even(num):
  return num % 2 == 0
is_even(10) # True


UsageError: Line magic function `%paste` not found.


In [188]:
%magic